In [ ]:
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import requests
import os

WIKI_URL = "https://en.wikipedia.org/w/api.php"
MODELS = [{"name": "gpt-4.1-mini"}, {"name": "gpt-5-mini"}]


In [ ]:

load_dotenv()

In [ ]:
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

if openai_key:
    print("OpenAI API key loaded successfully.")
else:
    print("OpenAI API key not found.")


In [ ]:
openai = OpenAI(api_key=openai_key)

In [ ]:
class Wiki:
    def __init__(self):
        self.cache = {}
        self.api_url = WIKI_URL
        self.number_of_results = 5

    def search_wiki(self, search_prompt):
        # First, search for article titles
        search_params = {
            "action": "opensearch",
            "format": "json",
            "search": search_prompt,
            "limit": 1  # Get only the top result
        }
        
        headers = {
            "User-Agent": "WikiAssistant/1.0 (Educational Project; Python/requests)"
        }
        
        search_response = requests.get(self.api_url, params=search_params, headers=headers)
        search_results = search_response.json()
        
        # Check if we got any results
        if not search_results[1]:
            return f"No Wikipedia articles found for '{search_prompt}'"
        
        # Get the first article title
        article_title = search_results[1][0]
        
        # Now get the clean content using the API
        return self.get_wiki_page(article_title)
    
    def get_wiki_page(self, titles):
        # Wikipedia API endpoint
        params = {
            "action": "query",
            "format": "json",
            "titles": titles,
            "prop": "extracts",
            "exintro": False,
            "explaintext": True
        }
        
        # Wikipedia requires a User-Agent header
        headers = {
            "User-Agent": "WikiAssistant/1.0 (Educational Project; Python/requests)"
        }

        # Make the API call
        response = requests.get(WIKI_URL, params=params, headers=headers)
        
        # Check if request was successful
        if response.status_code != 200:
            return f"Error: API returned status code {response.status_code}\n{response.text[:500]}"
        
        data = response.json()

        # Extract the page content
        pages = data.get("query", {}).get("pages", {})
        if not pages:
            return "No pages found in response"
        
        page_id = list(pages.keys())[0]
        page = pages[page_id]
        
        # Check if page exists
        if "missing" in page:
            return f"Article '{titles}' not found on Wikipedia"
        
        # Get the extract
        content = page.get("extract", "")
        
        if not content:
            return f"No content available for '{titles}'"
        
        return content

In [ ]:
system_prompt = """
You are Wiki-the-assistant, an AI assistant that provides concise and accurate information
 from Wikipedia articles.
"""

wiki_search_tool = {
    "type": "function",
    "function": {
        "name": "search_wiki",
        "description": "Search Wikipedia for articles related to the user's query.",
        "parameters": {
            "type": "object",
            "properties": {
                "search_prompt": {
                    "type": "string",
                    "description": "The search term to look up on Wikipedia."
                }
            },
            "required": ["search_prompt"],
            "additionalProperties": False
        }
    }
}

In [ ]:
wiki = Wiki()
result = wiki.search_wiki("XGBoost")

In [ ]:
result

In [ ]:
tools = [{"definition": wiki_search_tool, "function": wiki.search_wiki}]

def chat(message, history, model_name):
    messages = [{"role":"system", "content":system_prompt}] + \
        [{"role":h["role"], "content":h["content"]} for h in history] + \
        [{"role":"user", "content":message}]

    response = openai.chat.completions.create(model=model_name, messages=messages, tools=[tool["definition"] for tool in tools])
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        messages.append(message)
        used_tools = [tool_call.function.name for tool_call in response.choices[0].message.tool_calls]
        print(f"Calling tools: {', '.join(used_tools)}...")
        yield f"Calling tools: {', '.join(used_tools)}..."
        responses = handle_tool_calls(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=model_name, messages=messages)
    yield response.choices[0].message.content

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        tool_args = eval(tool_call.function.arguments)  # Parse JSON string
        for tool in tools:
            if tool["definition"]["function"]["name"] == tool_name:
                tool_response = tool["function"](**tool_args)
                print(tool_response)
                responses.append({
                    "role": "tool", 
                    "name": tool_name, 
                    "content": tool_response, 
                    "tool_call_id": tool_call.id
                })
    return responses

In [ ]:
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Wiki-the-assistant",
    description="An AI assistant that provides information from Wikipedia articles.",
    additional_inputs=[
        gr.Dropdown(
            choices=[model['name'] for model in MODELS],
            value=MODELS[0]['name'],
            label="Select Model"
        )
    ]
).launch()